In [1]:
import os
os.chdir("..")

import pathpy as pp
from pathpy import Network
import matplotlib.pyplot as plt

from typing import TYPE_CHECKING, Dict, List, Tuple, Union, Any
import scipy.sparse
from scipy.sparse import linalg as spl
import numpy as np

from collections import defaultdict # think these two are used to iterate through dicts
from collections.abc import Iterable
from __future__ import annotations
from typing import TYPE_CHECKING, Union, Dict, Tuple

import my_functions

In [2]:
# from paths dataset - 1% sample to start
flight_paths= pp.Paths.read_file("Data/US flights 2011/US flights od.ngram", separator=',', frequency=True)
m1 = pp.Network.from_paths(flight_paths)
m2 = pp.HigherOrderNetwork(flight_paths, k=2)

2021-04-07 19:04:10 [Severity.INFO]	Reading ngram data ... 
2021-04-07 19:04:10 [Severity.INFO]	finished. Read 358 paths with maximum length 7
2021-04-07 19:04:10 [Severity.INFO]	Calculating sub path statistics ... 
2021-04-07 19:04:10 [Severity.INFO]	finished.


# Core-periphery for $\mathcal{M}_1$ network
Quite helpful: https://github.com/IngoScholtes/kdd2018-tutorial/blob/master/solutions/1_2_pathpy.py <br>
For directed and weighted networks. But needs to be connected because uses stationary distributions. For now take reduced connected network and look at teleportation later if possible. <br>
Node strength $s_i$ is the sum of the weights of the links connected to that node
1. Start at the node with the minimum total node strength $s^{in}_i+s^{out}_i$ and set $S=v_i$
2. Add a node to $S$ such that it creates the smallest increase in persistence probability \begin{equation}
\alpha_S = \frac{\sum_{i,j \in S}p_i^*T_{ij} }{\sum_{i \in S}p_i^*}
\end{equation}
3. Then the $\alpha$-periphery is the set of nodes $S_{\alpha}$ satisfying $\alpha_i \leq \alpha$ and you can tune by $\alpha$
Choose uniformly if you get have multiple nodes with the same strength etc <br> <br>

    functions moved to my_functions module

# Core-periphery for $\mathcal{M}_2$ network

## adapted

In [4]:
def get_node_strengths_m2(net):
    inw_list = list(zip(*net.node_properties('inweight')))[0]
    outw_list = list(zip(*net.node_properties('outweight')))[0]
    strengths = np.array(inw_list) + np.array(outw_list)
    return strengths

def remove_hanging_m2(net,method="cut"):
    '''Remove hanging nodes and returns network, need to this becuase otherwise the transition matrix
    T will be ill-defined'''
    if method == "teleport":
        return "no method defined for this yet"
    
    elif method == "cut":
        out_degrees = np.array(net.node_properties('outdegree'))
        
        if (out_degrees == 0).any():
            ind = np.where(out_degrees == 0)[0] 
            hanging_nodes = []
            for i in ind: hanging_nodes.append(list(net.nodes.keys())[i])
            for node in hanging_nodes : net.remove_node(node)
            
        return net
    
    else:
        return "no method defined for this"
    
def alpha_S(ind, T_t, ps_t):
    '''Calculate the persistence probability of a set of nodes S 
    ind : list of indices
    T_t : sparse transposed transition matrix
    ps_t = stationary probability column vector'''
    T_t = T_t[ind,:][:,ind]
    ps_t = ps_t[ind]
    return T_t.dot(ps_t).sum()/ps_t.sum()

## in progress

In [5]:
net = m2

In [14]:
net = remove_hanging_m2(net)
print(net.ecount())

336


In [15]:
A = net.adjacency_matrix(weighted=True)
T_t = net.transition_matrix()
T = T_t.transpose()
N = np.shape(A)[0]

out_degrees = np.array(net.node_properties('outdegree'))
node_strengths = get_node_strengths_m2(net) # problematic
assert ((out_degrees > 0).all()), "Network must be ergodic."




In [16]:
net

In [17]:
# get stationary distribution of first-order Markov process
# https://stackoverflow.com/questions/30966881/python-scipy-sparse-linalg-eigs-giving-different-results-for-consecutive-calls
# Implicitly Restarted Arnoldi Method: occasionally gives negative results???
# it sometimes identifies the Fiedler vector as dominant eigenvector which means spectral gap must be small
# make statement that is lambda_1>lambda_2 then rerun
v11 = -1
while v11 < 0:
    _,ps = scipy.sparse.linalg.eigs(T,k=1,which='LM',tol=1e-6,maxiter=N*100) # check L&R
    ps_t = ps.real
    v11 = ps_t[0]
    #fiedler_vec = evecs[:,1].real
    #spectral_gap = lambda_1-lambda_2
    
# pretending this is working for now and moving on so I don't lose my mind

In [18]:
# get indices for node names (quick fix for now)
nodes_ind = np.arange(0,N)
node_dict = {}
coreness = {}
i=0
for node in net.nodes.keys():
    node_dict.update({i:node})
    coreness.update({node:0.0}) # initialise to zero
    i=i+1

In [19]:
# single run
#print(r/R)
# calculate node strengths
i_min = np.where(node_strengths == node_strengths.min())[0] 
i_min = np.random.choice(i_min,1)[0] # randomly choose one if there's multiple mins
coreness[node_dict[i_min]] = coreness[node_dict[i_min]] + 0.0
s0 = list(net.nodes.keys())[i_min] # node to initalise CP algorithm

# (greedy) algorithm to test which node will create smallest increase in alpha_S
S = [i_min]
alpha = alpha_S(S,T_t,ps_t) # alpha_S is zero for any single node
nodes = np.delete(nodes_ind,i_min)

while (len(nodes)>0):
    alpha_test = np.empty(N)
    alpha_test[:] = np.nan

    # calculate alpha increase for each node
    for node in nodes:
        S_test = S + [node]
        alpha_test[node] = alpha_S(S_test,T_t,ps_t) 

    # choose minimum alpha for this step
    node_min = np.where(alpha_test == np.nanmin(alpha_test))[0] 
    node_min = np.random.choice(node_min,1)[0]
    alpha = alpha + alpha_test[node_min]
    coreness[node_dict[node_min]] = coreness[node_dict[node_min]] + alpha_test[node_min]

    S = S + [node_min]
    nodes = nodes[nodes!=node_min]

In [20]:
for node, val in coreness.items():
        net.nodes[node]['coreness']=val

In [21]:
my_functions.print_coreness(net,coreness)

In [28]:
m2.nodes

coreness_map = {}
for m2_node in m2.nodes.keys():
    coreness_map.update({m2_node[0:3]:coreness[m2_node]})

In [30]:
for node, val in coreness_map.items():
    m1.nodes[node]['coreness']=val

In [38]:
coreness_map

{'DEN': 0.0,
 'AUS': 0.0200085230629285,
 'BOI': 0.0,
 'LAX': 0.0,
 'LIH': 0.0,
 'KOA': 0.0,
 'MCO': 0.45884984517047805,
 'CLE': 0.0,
 'SFO': 0.0,
 'LGA': 0.0,
 'ORD': 0.0,
 'PHX': 0.0,
 'SLC': 0.14161107191994363,
 'SBP': 0.15290567697787963,
 'MSP': 0.0,
 'SAN': 0.8200631471179638,
 'HNL': 0.0003548946438547728,
 'EGE': 0.0,
 'BOS': 0.0,
 'ELP': 0.0,
 'PDX': 0.13585006842146813,
 'SEA': 0.0,
 'PHL': 0.6328981949478474,
 'OGG': 0.0,
 'ATL': 0.0,
 'SNA': 0.0,
 'PSC': 0.09959201790885412,
 'SJC': 0.363105321731264,
 'GEG': 0.15844225005219775,
 'OAK': 0.0,
 'CLD': 0.0,
 'RNO': 0.0,
 'BZN': 0.0,
 'IAH': 0.0,
 'DTW': 0.0,
 'RDM': 0.0,
 'SBA': 0.32547554287535624,
 'IAD': 0.0,
 'SMF': 0.0,
 'CVG': 0.0,
 'ONT': 0.0,
 'JFK': 0.0,
 'MSY': 0.14729577235656519,
 'MMH': 0.0,
 'PIT': 0.0,
 'STL': 0.0,
 'TUS': 0.0,
 'BWI': 0.0,
 'LAS': 0.0,
 'MIA': 0.0,
 'LGB': 0.0,
 'EWR': 0.0,
 'SAT': 0.0,
 'BUR': 0.31579621053064044,
 'PSP': 0.0,
 'MRY': 0.0,
 'CLT': 0.0,
 'BNA': 0.0,
 'DAL': 0.003697648890060

In [36]:
alpha_c=1e-3
col1='lightskyblue'
col2='darkorange'
style={}
style['node_color']={v:col1 if u < alpha_c else col2 for v,u in coreness_map.items()}
style['force_charge']={v: -20 if u<alpha_c else -20 for v,u in coreness_map.items()}
pp.visualisation.plot(m1, **style)

    # to check T working as it should 
    ones = np.ones(4) # generalise to get size automatically later
    ones
    T.dot(ones) # check this is satisfied => should have an eigenvalue unity now
    
    # laplacian for later
    L = net.laplacian_matrix(weighted=True)
    Ldense = L.todense()

# Centrality functions

In [2]:
## from pathpy repo

def eigenvector_centrality(network: Network, # arg name and type (didn't know you could do this)
                           weight: Union[str, bool, None] = None,
                           alpha=0.85,
                           **kwargs: Any) -> dict:
    """Calculates the eigenvector centrality of all nodes.

    Parameters
    ----------
    network : Network

        The :py:class:`Network` object that contains the network

    Examples
    --------
    Compute eigenvector centrality in a simple network

    >>> import pathpy as pp
    >>> net = pp.Network(directed=True)
    >>> net.add_edge('a', 'x')
    >>> net.add_edge('x', 'b')
    >>> c = pp.algorithms.centralities.eigenvector_centrality(net)
    >>> c['a']
    1
    """
    evcent: dict = dict()
        
    A = network.adjacency_matrix(weighted=True, transposed=True) # changed this for my version
    N = A.shape[0] # total nodes
    I = scipy.sparse.identity(N) # identity
    ev = spl.inv(I - alpha*A).dot(ones)
    
    if kwargs:
        _, ev = spl.eigs(A, k=1, which='LM', **kwargs) # which='LM' means find biggest, kwargs if changing eigenvec to test
    else:
        _, ev = spl.eigs(A, k=1, which='LM') # only returns vec for stationary distributino
    ev = ev.reshape(ev.size, ) # makes it a column vector I think
    S = np.sum(ev)
    
    for v, deg in network.nodes.items():
        evcent[v] = np.real(ev[network.nodes.index[v.uid]]/S) 
    for v in network.nodes:
        evcent[v.uid] = np.real(ev[network.nodes.index[v.uid]]/S) 
    return evcent


# so last few lines: take eigenvector corresponding to largest eigenvalue (1) of adjacency matrix of net
# reshape this vector to a column(?) vector. evcent is an empty dictionary. Then define for each key 1.uid 
# (not sure what uid stands for...) the values is the real part of the stationary probability averaged over 
# all the stationary probabilities and these will always sum to 1

## Trying to come generalise PageRank like in $(181)$

\begin{equation}
p^* = \frac{1-\alpha}{N}\begin{pmatrix}1 & ... & 1 \end{pmatrix} \begin{pmatrix} I - \alpha T \end{pmatrix}^{-1}
\end{equation}

My main issues are keeping it efficient, use scipy.sparse etc. and also keeping track of the indices...except now that I think about it all that matters are the indices of $p^*=(p_1^*...p_N^*)$ so it's ok. <br><br>
T is defined wrong here, it is meant to be $T_{ij} = \frac{A_{ij}}{k_i^{\text{out}}}$., just going to assume $A_{ij}$ defines a path from $i \to j$ but might need to change this. Hence each row will need to be divide by it's sum to get $T$.

In [169]:

def my_pagerank(network,alpha = 0.85):

    node_names = list(network.nodes.keys())

    A = network.adjacency_matrix(weighted=True, transposed=False) # unsure about transpose
    N = A.shape[0] # total nodes
    I = scipy.sparse.identity(N) # identity
    ones = np.ones(N)
    Ap = spl.inv(I - alpha*A)
    ev = (1-alpha)*Ap.dot(ones)/N
    S = sum(ev)
    ev = ev/S # normalised for some reason - might not be correct I need to check
    evcents = list(zip(node_names, zip(ev)))
    
    return evcents

I should be looking at PageRank because that's for directed networks (https://stats.stackexchange.com/questions/176874/pagerank-vs-eigenvector-centrality) PageRank uses in-degree of nodes specifically so is a special case of eigenvector centrality.

In [168]:
A = network.adjacency_matrix(weighted=True, transposed=False) # unsure about transpose
print(A)

  (0, 1)	1.0
  (0, 3)	1.0
  (1, 2)	1.0
  (3, 1)	1.0


In [197]:
out_deg = A.sum(1)
out_deg = np.squeeze(np.asarray(out_deg))

In [200]:
print(A)

  (0, 1)	1.0
  (0, 3)	1.0
  (1, 2)	1.0
  (3, 1)	1.0


In [201]:
out_deg = scipy.sparse.spdiags(out_deg, 0, out_deg.size, out_deg.size)
out_deg.dot(A)

/usr/local/lib/python3.9/site-packages/scipy/sparse/sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


TypeError: no supported conversion for types: (dtype('O'),)

In [165]:
network

In [164]:
my_pagerank(network)

[('a', (0.4674229310154322,)),
 ('x', (0.1816998760021117,)),
 ('b', (0.09821614919033063,)),
 ('c', (0.25266104379212556,))]

In [160]:
pp.algorithms.centralities.pagerank(network)

{'a': 0.12045209234069676,
 'x': 0.3175410567582492,
 'b': 0.39036291569760717,
 'c': 0.1716439352034471}

My calculated results are different to the results of the PageRank function, but which is more accurate? <br>
Neither perform very well tbh. It's clear that b's centrality is too high because it has no out-degree ( can't see function so don't know if that transition matrix has been set up yet, I must fix that actually).

In [162]:
network

In [94]:
A = net.adjacency_matrix()
print(A)

  (0, 1)	1.0
  (1, 2)	1.0


In [19]:
at = a.reshape(a.size,)
at

array([ 0.00000000e+000, -2.68678569e+154,  1.48219694e-323,
        0.00000000e+000,  0.00000000e+000,  4.17201348e-309])

In [20]:
at.shape

(6,)